In [1]:
import numpy as np
import cv2

Set some parameters for Shi-Thomasi 

In [2]:
# Parameters for Shi-Tomasi corner detection (good features to track paper)
corner_track_params = dict(maxCorners = 10,
                           qualityLevel = 0.3,
                           minDistance = 7,
                           blockSize = 7 )

goodFeaturesToTrack(image, maxCorners, qualityLevel, minDistance[, corners[, mask[, blockSize[, useHarrisDetector[, k]]]]]) -> corners
.   @brief Determines strong corners on an image.
.   
.   The function finds the most prominent corners in the image or in the specified image region, as
.   described in @cite Shi94
.   
.   -   Function calculates the corner quality measure at every source image pixel using the
.       #cornerMinEigenVal or #cornerHarris .
.   -   Function performs a non-maximum suppression (the local maximums in *3 x 3* neighborhood are
.       retained).
.   -   The corners with the minimal eigenvalue less than
.       \f$\texttt{qualityLevel} \cdot \max_{x,y} qualityMeasureMap(x,y)\f$ are rejected.
.   -   The remaining corners are sorted by the quality measure in the descending order.
.   -   Function throws away each corner for which there is a stronger corner at a distance less than
.       maxDistance.
.   
.   The function can be used to initialize a point-based tracker of an object.
.   
.   @note If the function is called with different values A and B of the parameter qualityLevel , and
.   A \> B, the vector of returned corners with qualityLevel=A will be the prefix of the output vector
.   with qualityLevel=B .
.   
.   @param image Input 8-bit or floating-point 32-bit, single-channel image.
.   @param corners Output vector of detected corners.
.   @param maxCorners Maximum number of corners to return. If there are more corners than are found,
.   the strongest of them is returned. `maxCorners <= 0` implies that no limit on the maximum is set
.   and all detected corners are returned.
.   @param qualityLevel Parameter characterizing the minimal accepted quality of image corners. The
.   parameter value is multiplied by the best corner quality measure, which is the minimal eigenvalue
.   (see #cornerMinEigenVal ) or the Harris function response (see #cornerHarris ). The corners with the
.   quality measure less than the product are rejected. For example, if the best corner has the
.   quality measure = 1500, and the qualityLevel=0.01 , then all the corners with the quality measure
.   less than 15 are rejected.
.   @param minDistance Minimum possible Euclidean distance between the returned corners.
.   @param mask Optional region of interest. If the image is not empty (it needs to have the type
.   CV_8UC1 and the same size as image ), it specifies the region in which the corners are detected.
.   @param blockSize Size of an average block for computing a derivative covariation matrix over each
.   pixel neighborhood. See cornerEigenValsAndVecs .

In [3]:
lk_params = dict(winSize= (200,200) , maxLevel= 2 , criteria=(cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS ,10, 0.03))
#- Window size (200,200) smaller size --> more sensitive to noise but can not catch large optical motion 

#- maxlevel --> lucase-kanade use pyramid levels to subsample the image or frame  --> =2 == 1/4 resolution
#    https://en.wikipedia.org/wiki/Pyramid_%28image_processing%29

# criteria has two here - the max number (10 above) of iterations and epsilon (0.03 above).
#- More iterations means a more exhaustive search for the points in current frame Vs prev. frame(How many iterations will take looking for these points)
#- and a smaller epsilon means that we will finish earlier. 
#- These are primarily useful in exchanging speed vs accuracy, but mainly stay the same.

In [4]:
cap = cv2.VideoCapture(0)

ret , prev_frame = cap.read()

prev_gray= cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# the points we want to track 

#Grab using corner_tracking_parameters some goodfeaturetotrack 

# Grab top 10 corners and track those

prevPts= cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    
    ret , frame = cap.read()

    frame_gray= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    nextPts , status , err = cv2.calcOpticalFlowPyrLK(frame_gray, prev_gray, prevPts , None , **lk_params)
    
    good_new= nextPts[status==1]
    
    good_prev= prevPts[status==1]
    
     # Use ravel to get points to draw lines and circles
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new =new.ravel() 
        x_prev,y_prev = prev.ravel()
        
        # Lines will be drawn using the mask created from the first frame
        mask = cv2.line(mask, pt1=(int(x_new),int(y_new)),pt2=(int(x_prev),int(y_prev)),color=(0,255,0),thickness= 3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame,(int(x_new),int(y_new)),8,(0,0,255),-1)
    
    # Display the image along with the mask we drew the line on.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()



In [5]:
good_new

array([[363.86227, 264.00778]], dtype=float32)

In [6]:
good_prev

array([[363.83908, 264.21875]], dtype=float32)

In [7]:
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        print (x_new, y_new)

363.86227 264.00778


In [8]:
x=type(x_new)
x

numpy.float32